EXP-15 : Design a dashboard layout that includes an interactive component to 15 display sales, profits, and customer count based on a selected date range.
OUTPUT : A functional dashboard where users can filter by date range and view dynamic visualization

1. Import necessary libraries (Dash, Plotly, Pandas, Dash Table).
2. Load and preprocess the dataset (parse dates, calculate profit).
3. Group data by relevant categories (Date, Region, Product Category).
4. Create a Dash app layout with components (title, date picker, graphs, table).
5. Set up callback functions to filter data and update graphs and table based on selected date range.
6. Run the Dash app for interactivity and dynamic updates.

In [8]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output
import dash_table

# Load the dataset (replace with the correct file path)
df = pd.read_csv('sales_data.csv')

# Convert 'Date' to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Aggregate data for sales, profit, and customer count
# Let's assume the "Profit" is calculated as "Sales" - some cost (assuming no cost data, use a basic method)
df['Profit'] = df['Sales'] * 0.2  # Assuming a 20% profit margin

# Group the data by Date, Region, and Product Category
grouped_df = df.groupby(['Date', 'Region', 'Product Category']).agg({
    'Sales': 'sum',
    'Profit': 'sum'
}).reset_index()

# Create the Dash app
app = dash.Dash(__name__)

# Layout of the dashboard
app.layout = html.Div([
    html.H1("Sales Dashboard"),
    
    # Date Picker Range for filtering data
    dcc.DatePickerRange(
        id='date-picker-range',
        start_date=df['Date'].min().date(),
        end_date=df['Date'].max().date(),
        display_format='YYYY-MM-DD',
        style={'padding': '10px'}
    ),
    
    # Graph displaying Sales over time
    dcc.Graph(id='sales-graph'),
    
    # Graph displaying Profit over time
    dcc.Graph(id='profit-graph'),
    
    # Table showing aggregated data for the selected date range
    dash_table.DataTable(
        id='sales-table',
        columns=[{"name": col, "id": col} for col in grouped_df.columns],
        style_table={'height': '350px', 'overflowY': 'auto'}
    )
])

# Callbacks to update the graphs and table based on the selected date range
@app.callback(
    [Output('sales-graph', 'figure'),
     Output('profit-graph', 'figure'),
     Output('sales-table', 'data')],
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)
def update_dashboard(start_date, end_date):
    # Filter data based on the selected date range
    filtered_df = grouped_df[(grouped_df['Date'] >= start_date) & (grouped_df['Date'] <= end_date)]
    
    # Create the sales graph
    sales_fig = px.line(filtered_df, x='Date', y='Sales', color='Product Category', title='Sales Over Time')

    # Create the profit graph
    profit_fig = px.line(filtered_df, x='Date', y='Profit', color='Product Category', title='Profit Over Time')

    # Convert filtered dataframe to a list of dictionaries for the table
    table_data = filtered_df.to_dict('records')
    
    return sales_fig, profit_fig, table_data

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True,port=8052)
